In [ ]:
import pandas as pd
import pathlib
import win32com.client as win32
import locale
import math

# Leitura dos arquivos:
email_df = pd.read_excel(r'Base de Dados\Emails.xlsx')
lojas_df = pd.read_csv(r'Base de Dados\Lojas.csv', sep=';', encoding='latin-1')
vendas_df = pd.read_excel(r'Base de Dados\Vendas.xlsx')

completo_df = vendas_df.merge(lojas_df, on='ID Loja')

display(completo_df)

In [ ]:
#Verificando os dados:
completo_df.info()

In [ ]:
#Separando o dataframe de cada loja:
dic_lojas = {}

for loja in lojas_df['Loja']:
    dic_lojas[loja] = completo_df[completo_df['Loja'] == loja]

In [ ]:
#Pegando o último dia da planilha:
dia_indicador = completo_df['Data'].max()

In [ ]:
#Salvando os dataframes de cada loja em sua pasta de Backup:
caminho = pathlib.Path('Backup Arquivos Lojas')

for loja in dic_lojas:
    pasta_loja = caminho / f'{loja}'
    pasta_loja.mkdir(exist_ok=True)

    nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'
    local_arquivo = pasta_loja / nome_arquivo
    dic_lojas[loja].to_excel(local_arquivo)

In [ ]:
#Metas:
META_FATURAMENTO_DIA = 1000
META_FATURAMENTO_ANO = 1650000
META_QTDE_PRODUTOS_DIA = 4
META_QTDE_PRODUTOS_ANO = 120
META_TICKETMEDIO_DIA = 500
META_TICKETMEDIO_ANO= 500

In [ ]:
# Define a localidade como Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

def formatar_numero(numero):
    '''Formata os valores para o padrão de moeda PT-BR'''
    return locale.format_string('%.2f', numero, grouping=True)

def verificar_cor(indicativo, meta):
    if indicativo >= meta:
        return 'green'
    else:
        return 'red'

for loja in dic_lojas:

    vendas_loja = dic_lojas[loja]
    vendas_loja_dia = vendas_loja[vendas_loja['Data']==dia_indicador]
    
    #Faturamento:
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #Diversidade de Produtos:
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    #Ticket Médio:
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    if math.isnan(ticket_medio_ano):
        ticket_medio_ano = 0
    
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    if math.isnan(ticket_medio_dia):
        ticket_medio_dia = 0

    #Criando e Enviando e-mail:
    outlook = win32.Dispatch('outlook.application')

    nome = email_df.loc[email_df['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = email_df.loc[email_df['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year} - Loja {loja}'

    #Definindo as cores dos indicadores:
    cor_fat_ano = verificar_cor(faturamento_ano, META_FATURAMENTO_ANO)
    cor_fat_dia = verificar_cor(faturamento_dia, META_FATURAMENTO_DIA)
    cor_qtde_ano = verificar_cor(qtde_produtos_ano, META_QTDE_PRODUTOS_ANO)
    cor_qtde_dia = verificar_cor(qtde_produtos_dia, META_QTDE_PRODUTOS_DIA)
    cor_ticket_ano = verificar_cor(ticket_medio_ano, META_TICKETMEDIO_ANO)
    cor_ticket_dia = verificar_cor(ticket_medio_dia, META_TICKETMEDIO_DIA)

    #Corpo do e-mail:
    mail.HTMLBody = f'''
    <p>Bom dia, prezado(a) Sr.(a) {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${formatar_numero(faturamento_dia)}</td>
        <td style="text-align: center">R${formatar_numero(META_FATURAMENTO_DIA)}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{META_QTDE_PRODUTOS_DIA}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${formatar_numero(ticket_medio_dia)}</td>
        <td style="text-align: center">R${formatar_numero(META_TICKETMEDIO_DIA)}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${formatar_numero(faturamento_ano)}</td>
        <td style="text-align: center">R${formatar_numero(META_FATURAMENTO_ANO)}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{META_QTDE_PRODUTOS_ANO}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${formatar_numero(ticket_medio_ano)}</td>
        <td style="text-align: center">R${formatar_numero(META_TICKETMEDIO_ANO)}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Diego Monutti</p>
    '''

    #Add anexos:
    attachment = pathlib.Path.cwd() / local_arquivo
    mail.Attachments.Add(str(attachment))
    mail.Send()
    print(f'E-mail da Loja {loja} enviado')

In [ ]:
#Criar e salvar Ranking de Lojas para Diretoria:

#Faturamento anual:
faturamento_lojas = completo_df.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

#Salvar o ranking de faturamento anual:
nome_arquivo_ano = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Anual.xlsx'
caminho_arquivo_ano = caminho / nome_arquivo_ano
faturamento_lojas_ano.to_excel(caminho_arquivo_ano)

#Faturamento último dia:
vendas_dia = completo_df[completo_df['Data']==dia_indicador]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()

#Incluir no ranking as lojas que não venderam no último dia:
faturamento_lojas_dia = faturamento_lojas_dia.merge(lojas_df[['Loja']], on='Loja', how='right')
faturamento_lojas_dia.fillna(0, inplace=True)
faturamento_lojas_dia = faturamento_lojas_dia.set_index('Loja')

faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

#Salvar o ranking de faturamento do último dia:
nome_arquivo_dia = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Anual.xlsx'
caminho_arquivo_dia = caminho / nome_arquivo_dia
faturamento_lojas_dia.to_excel(caminho_arquivo_dia)

In [ ]:
#Listando as lojas que não venderam no último dia:
lojas_sem_venda = faturamento_lojas_dia[faturamento_lojas_dia['Valor Final'] == 0].index.tolist()

#Definindo o ranking apenas com as lojas que tiveram vendas:
lojas_com_venda = faturamento_lojas_dia[faturamento_lojas_dia['Valor Final'] != 0]

#Definindo as melhores e piores lojas:
loja_menor_faturamento_dia = lojas_com_venda['Valor Final'].idxmin()
menor_faturamento_dia = lojas_com_venda['Valor Final'].min()

loja_maior_faturamento_dia = faturamento_lojas_dia['Valor Final'].idxmax()
maior_faturamento_dia = faturamento_lojas_dia['Valor Final'].max()

loja_menor_faturamento_ano = faturamento_lojas_ano['Valor Final'].idxmin()
menor_faturamento_ano = faturamento_lojas_ano['Valor Final'].min()

loja_maior_faturamento_ano = faturamento_lojas_ano['Valor Final'].idxmax()
maior_faturamento_ano = faturamento_lojas_ano['Valor Final'].max()


In [ ]:
#Enviar e-mail para diretoria:
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = email_df.loc[email_df['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year}'
mail.Body = f'''
Prezados membros da Diretoria, bom dia,

Segue o resultado de desempenho das lojas com base no dia {dia_indicador.day}/{dia_indicador.month}/{dia_indicador.year}:

Melhor loja do Dia em Faturamento: Loja {loja_maior_faturamento_dia} com Faturamento de R${formatar_numero(maior_faturamento_dia)}
Pior loja do Dia em Faturamento: Loja {loja_menor_faturamento_dia} com Faturamento de R${formatar_numero(menor_faturamento_dia)}

Melhor loja do Ano em Faturamento: Loja {loja_maior_faturamento_ano} com Faturamento R${formatar_numero(maior_faturamento_ano)}
Pior loja do Ano em Faturamento: Loja {loja_menor_faturamento_ano} com Faturamento R${formatar_numero(menor_faturamento_ano)}

Em anexo envio os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Diego Monutti
'''

# Anexos (pode colocar quantos quiser):
attachment  = pathlib.Path.cwd() / caminho_arquivo_dia'
mail.Attachments.Add(str(attachment))
attachment  = pathlib.Path.cwd() / caminho_arquivo_ano'
mail.Attachments.Add(str(attachment))

mail.Send()
print('E-mail da Diretoria enviado')